In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import snapshot_download
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "Qwen/Qwen2-0.5B-Instruct"
output_dir ="assets\\Qwen2_0.5B_Instruct"

In [3]:
# snapshot_download(repo_id=model_name,local_dir=output_dir)


In [4]:
model = AutoModelForCausalLM.from_pretrained(
    output_dir,
    device_map=device
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [5]:
print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))

torch.cuda.memory_allocated: 1.840416GB
torch.cuda.memory_reserved: 2.371094GB
torch.cuda.max_memory_reserved: 2.371094GB


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
type(tokenizer)

transformers.models.qwen2.tokenization_qwen2_fast.Qwen2TokenizerFast

In [8]:
prompt = "Give me a short introduction to large language model."

In [9]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

In [10]:
messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': 'Give me a short introduction to large language model.'}]

In [11]:
text = tokenizer.apply_chat_template(
    [messages,messages],
    tokenize=False,
    add_generation_prompt=True
)

In [12]:
text

['<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nGive me a short introduction to large language model.<|im_end|>\n<|im_start|>assistant\n',
 '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nGive me a short introduction to large language model.<|im_end|>\n<|im_start|>assistant\n']

In [13]:
model_inputs = tokenizer([text,text], return_tensors="pt").to(model.device)


In [14]:
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [15]:
print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))

torch.cuda.memory_allocated: 1.848357GB
torch.cuda.memory_reserved: 2.388672GB
torch.cuda.max_memory_reserved: 2.388672GB
